# Using Monte Carlo Tree Search for your Fantasy Football draft
My blog post about this notebook can be found on [Medium](https://medium.com/@ykeuter/using-monte-carlo-tree-search-for-your-fantasy-football-draft-6509b78a1c20).

In [1]:
class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.turns = turns
        self.freeagents = freeagents
        self.playerJustMoved = playerjm

In [2]:
class NflPlayer:
    def __init__(self, name, team, position, points):
        self.name = name
        self.team = team
        self.position = position
        self.points = points
        
    def __repr__(self):
        return "|".join([self.name, self.team, self.position, str(self.points)])

In [3]:
import numpy as np

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("QB"): [.6, .4],
        ("WR"): [.7, .7, .4, .2],
        ("RB"): [.7, .7, .4, .2],
        ("TE"): [.6, .4],
        ("RB", "WR", "TE"): [.6, .4],
        ("D"): [.6, .3, .1],
        ("K"): [.5, .2, .2, .1]
    }

    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
        
    return result
        
DraftState.GetResult = GetResult

In [4]:
def GetMoves(self):
    """ Get all possible moves from this state.
    """
    pos_max = {"QB": 2, "WR": 6, "RB": 6, "TE": 2, "D": 2, "K": 1}

    if len(self.turns) == 0: return []

    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    return moves

DraftState.GetMoves = GetMoves

In [5]:
def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

In [6]:
def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st

DraftState.Clone = Clone

In [7]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random

class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [8]:
import pandas as pd

nfl_players = pd.read_csv("nfl_players.csv", index_col=0)
freeagents = [NflPlayer(*p) for p in nfl_players.itertuples(index=False, name=None)]

num_competitors = 10
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

num_rounds = 16
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 1000
while state.GetMoves() != []:
    move = UCT(state, iterations)
    print(move, end=".")
    state.DoMove(move)

RB.RB.RB.WR.RB.RB.RB.RB.RB.RB.WR.WR.WR.WR.WR.RB.RB.RB.TE.WR.WR.TE.WR.RB.RB.WR.WR.WR.TE.WR.WR.TE.WR.WR.WR.QB.RB.QB.WR.WR.TE.QB.WR.TE.TE.RB.RB.RB.WR.RB.QB.RB.WR.WR.QB.RB.RB.WR.RB.RB.WR.D.TE.TE.RB.WR.TE.QB.WR.WR.RB.RB.RB.RB.TE.K.RB.WR.WR.RB.QB.WR.RB.QB.D.RB.QB.D.QB.D.K.D.QB.D.D.WR.WR.D.RB.D.WR.QB.WR.D.WR.WR.QB.D.K.D.TE.QB.WR.WR.QB.QB.RB.RB.WR.QB.RB.RB.WR.WR.RB.RB.RB.TE.WR.WR.WR.WR.WR.WR.WR.WR.K.K.WR.WR.TE.D.TE.WR.TE.K.D.RB.RB.QB.RB.D.TE.TE.D.WR.QB.QB.K.D.

In [9]:
pd.DataFrame({"Team " + str(i + 1): r for i, r in enumerate(state.rosters)})

,Team 1,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8,Team 9,Team 10
0,Le'Veon Bell|PIT|RB|351.1,Todd Gurley II|LAR|RB|325.8,David Johnson|ARI|RB|322.8,Antonio Brown|PIT|WR|320.3,Ezekiel Elliott|DAL|RB|293.1,Alvin Kamara|NO|RB|292.9,Saquon Barkley|NYG|RB|285.7,Dalvin Cook|MIN|RB|264.9,Melvin Gordon|LAC|RB|260.5,Kareem Hunt|KC|RB|258.9
1,A.J. Green|CIN|WR|265.6,Rob Gronkowski|NE|TE|237.2,Christian McCaffrey|CAR|RB|252.8,LeSean McCoy|BUF|RB|253.4,Leonard Fournette|JAX|RB|257.9,Michael Thomas|NO|WR|277.0,Odell Beckham Jr.|NYG|WR|277.8,Keenan Allen|LAC|WR|278.7,DeAndre Hopkins|HOU|WR|282.6,Julio Jones|ATL|WR|289.4
2,Davante Adams|GB|WR|252.9,Travis Kelce|KC|TE|216.8,Adam Thielen|MIN|WR|246.7,Devonta Freeman|ATL|RB|230.2,Jerick McKinnon|SF|RB|228.6,T.Y. Hilton|IND|WR|242.3,Larry Fitzgerald|ARI|WR|241.5,Tyreek Hill|KC|WR|240.3,Zach Ertz|PHI|TE|211.7,Demaryius Thomas|DEN|WR|235.4
3,Golden Tate|DET|WR|216.2,Josh Gordon|CLE|WR|219.6,Aaron Rodgers|GB|QB|313.3,Joe Mixon|CIN|RB|207.2,Tom Brady|NE|QB|316.2,Allen Robinson|CHI|WR|220.3,Stefon Diggs|MIN|WR|228.1,Doug Baldwin|SEA|WR|232.5,Greg Olsen|CAR|TE|194.3,Mike Evans|TB|WR|233.0
4,Delanie Walker|TEN|TE|191.7,Cam Newton|CAR|QB|303.9,JuJu Smith-Schuster|PIT|WR|214.5,Evan Engram|NYG|TE|182.2,Jordan Reed|WSH|TE|177.2,Jordan Howard|CHI|RB|200.1,Kenyan Drake|MIA|RB|199.4,Marshawn Lynch|OAK|RB|197.9,Alshon Jeffery|PHI|WR|212.3,Derrius Guice|WSH|RB|192.8
5,Alex Collins|BAL|RB|186.6,Ronald Jones II|TB|RB|187.6,Pierre Garcon|SF|WR|200.4,Royce Freeman|DEN|RB|187.7,Sony Michel|NE|RB|187.9,Russell Wilson|SEA|QB|296.5,Jarvis Landry|CLE|WR|210.0,Amari Cooper|OAK|WR|211.0,Rashaad Penny|SEA|RB|191.6,Carson Wentz|PHI|QB|298.3
6,Emmanuel Sanders|DEN|WR|199.7,Jaguars|JAX|D|123.0,Jack Doyle|IND|TE|167.1,Kyle Rudolph|MIN|TE|164.6,Derrick Henry|TEN|RB|185.4,Chris Hogan|NE|WR|197.2,Jimmy Graham|GB|TE|157.0,Andrew Luck|IND|QB|287.9,Michael Crabtree|BAL|WR|195.0,Marvin Jones Jr.|DET|WR|194.5
7,Chris Thompson|WSH|RB|163.8,Randall Cobb|GB|WR|189.4,Robert Woods|LAR|WR|189.9,Tarik Cohen|CHI|RB|166.0,Greg Zuerlein|LAR|K|151.6,Charles Clay|BUF|TE|144.0,Duke Johnson Jr.|CLE|RB|166.6,Dion Lewis|TEN|RB|171.3,Mark Ingram|NO|RB|174.3,Jay Ajayi|PHI|RB|175.0
8,Alex Smith|WSH|QB|281.0,Sammy Watkins|KC|WR|187.9,Tevin Coleman|ATL|RB|162.4,Deshaun Watson|HOU|QB|278.6,Eagles|PHI|D|113.3,Lamar Miller|HOU|RB|154.4,Matthew Stafford|DET|QB|274.6,Rams|LAR|D|108.3,Ben Roethlisberger|PIT|QB|274.4,Patriots|NE|D|108.0
9,Broncos|DEN|D|104.2,Marlon Mack|IND|RB|150.1,Chargers|LAC|D|104.6,Brandin Cooks|LAR|WR|186.7,Kelvin Benjamin|BUF|WR|187.1,Vikings|MIN|D|106.2,Texans|HOU|D|106.6,Kirk Cousins|MIN|QB|272.2,Ravens|BAL|D|107.8,Stephen Gostkowski|NE|K|145.6
